# https://blog.csdn.net/firparks/article/details/54563683

### 基于以下语料建立语言模型
Father read Holy Bible.

Mother read a text book.

He read a book by grandpa.

### 𝑝("Father read a book")= 0.06

In [6]:
sentence_ori="Father read Holy Bible. Mother read a text book. He read a book by grandpa."

In [7]:
sentence_test="Father read a book"

In [42]:
import jieba
from _overlapped import NULL

#将句子变为"BOSxxxxxEOS"这种形式
def reform(sentence):
    #如果是以“。”结束的则将“。”删掉
    if sentence.endswith("."):
        sentence=sentence[:-1]
    #添加起始符BOS和终止符EOS   
    sentence_modify1=sentence.replace(".", "EOSBOS")
    sentence_modify2="BOS"+sentence_modify1+"EOS"
    print('\nsentence_modify2:',sentence_modify2)
    return sentence_modify2


#分词并统计词频
def segmentation(sentence,lists,dicts=NULL):
    jieba.suggest_freq("BOS", True)
    jieba.suggest_freq("EOS", True)
    sentence = jieba.cut(sentence,HMM=False)
    format_sentence=",".join(sentence)
    #将词按","分割后依次填入数组word_list[]
    lists=format_sentence.split(",")
    #去除lists中的空格
    for i in lists:
        if ' ' in lists:
            lists.remove(' ')
    #统计词频，如果词在字典word_dir{}中出现过则+1，未出现则=1
    if dicts!=NULL:
        for word in lists:
            if word not in dicts:
                dicts[word]=1
            else:
                dicts[word]+=1               
    print('\nlists:',lists)
    print('\ndicts:',dicts)
    return lists


#比较两个数列，二元语法
def compareList(ori_list,test_list):
    #申请空间
    count_list=[0]*(len(test_list))
    #遍历测试的字符串
    for i in range(0,len(test_list)-1):
        #遍历语料字符串，且因为是二元语法，不用比较语料字符串的最后一个字符
        for j in range(0,len(ori_list)-2):                
            #如果测试的第一个词和语料的第一个词相等则比较第二个词
            if test_list[i]==ori_list[j]:
                if test_list[i+1]==ori_list[j+1]:
                    count_list[i]+=1
    return count_list


#计算概率    
def probability(test_list,count_list,ori_dict):
    flag=0
    #概率值为p
    p=1
    for key in test_list[0:-1]: 
        #数据平滑处理：加1法 ; 0表示未使用
        p*=(float(count_list[flag]+0)/float(ori_dict[key]))
        print(count_list[flag],ori_dict[key])
        flag+=1
    return p


if __name__ == "__main__":

    #语料句子
    sentence_ori="Father read Holy Bible. Mother read a text book. He read a book by grandpa."
    ori_list=[]
    ori_dict={}
    sentence_ori_temp=""

    #测试句子
    sentence_test="Father read a book"
    sentence_test_temp="" 
    test_list=[]
    count_list=[]
    p=0

    #分词并将结果存入一个list，词频统计结果存入字典
    sentence_ori_temp=reform(sentence_ori)
    ori_list=segmentation(sentence_ori_temp,ori_list,ori_dict)
    print('\nori_list',ori_list)

    sentence_test_temp=reform(sentence_test)
    test_list=segmentation(sentence_test_temp,test_list)
    print('\ntest_list',test_list)
    
    count_list=compareList(ori_list, test_list)
    print('\ncount_list',count_list)
    
    p=probability(test_list,count_list,ori_dict)

    print('\n"',sentence_test,'"出现的概率是：',p)




sentence_modify2: BOSFather read Holy BibleEOSBOS Mother read a text bookEOSBOS He read a book by grandpaEOS

lists: ['BOS', 'Father', 'read', 'Holy', 'Bible', 'EOS', 'BOS', 'Mother', 'read', 'a', 'text', 'book', 'EOS', 'BOS', 'He', 'read', 'a', 'book', 'by', 'grandpa', 'EOS']

dicts: {'BOS': 3, 'Father': 1, 'read': 3, 'Holy': 1, 'Bible': 1, 'EOS': 3, 'Mother': 1, 'a': 2, 'text': 1, 'book': 2, 'He': 1, 'by': 1, 'grandpa': 1}

ori_list ['BOS', 'Father', 'read', 'Holy', 'Bible', 'EOS', 'BOS', 'Mother', 'read', 'a', 'text', 'book', 'EOS', 'BOS', 'He', 'read', 'a', 'book', 'by', 'grandpa', 'EOS']

sentence_modify2: BOSFather read a bookEOS

lists: ['BOS', 'Father', 'read', 'a', 'book', 'EOS']

dicts: 0

test_list ['BOS', 'Father', 'read', 'a', 'book', 'EOS']

count_list [1, 1, 2, 1, 1, 0]
1 3
1 1
2 3
1 2
1 2

" Father read a book "出现的概率是： 0.05555555555555555
